Строк может быть только 18
Нужно написать Макрос, который ограничивает в Ворде кол-во строк до 18 включительно

In [1]:
import os
import pytesseract
from docx2pdf import convert
import subprocess
import time
import win32com.client  # Убедитесь, что этот импорт присутствует
import fitz  # PyMuPDF
from PIL import Image, ImageDraw, ImageFont

import tkinter as tk
from tkinter import messagebox
from docx import Document
import shutil

In [2]:
#  ----   Основные переменные для пользоваетля
input_dir = './input' # в этой папке лежат .docx-файлы лекций (ПОДДЕРЖИВАЕТ ТОЛЬКО DOCX)
pdf_dir = './pdf' # в эту папку скрипт сконвертирует .docx-файлы из input_dir. (это необходимо, т.к. скрипт видит страницы именно в pdf формате (а не в docx))

 # в этой папке скрипт создаст (для каждой лекции из input) свою подпапку
output_dir = './output'
# в папке output, в подпапке каждой лекции лежат две папки: to_combine и to_print

# В этой папке лежат сделанные из pdf картинки страниц текста
# эта папка лежит в output_dir.
to_combine_dir = './to_combine'                   #В папке to_combine должны быть картинки ТОЛЬКО с цифрами: 1.png, 2.png и т.д.

# В этой папке лежат объединенные специфическим образом для печати на А4 картинки из папки to_combine
# Это и есть готовые конспекты к печати

to_print_dir = './to_print'

# Загружаем шрифт нашего почерка@
font = ImageFont.truetype("./fonts/MyFont15.otf", size=67)

note_type = 'cells' #клеточки. note_type='lines' - лист в линейку

font_color = 'blue' #цвет шрифта

font_scale_y__coeff = 2 #обязательно int

# Задаем вертикальный шаг между строками
line_spacing = 110  # 55 - писать на каждой строке, 110 - пропускать одну строку между двумя строками текста

#Адрес файла запуска Microsoft Word
word_path = r"C:\Program Files\Microsoft Office\root\Office16\WINWORD.EXE" 




In [3]:
pdf_dir = './pdf'
output_dir = './output'
to_combine_dir = './to_combine'               
to_print_dir = './to_print'

# Очистить папки to_print_dir, to_combine_dir, pdf_dir ? По умолчанию - да
clear__pdf_dir = True
clear__to_print_dir = True
lear__to_combine_dir = True

#В папке output могут быть сколько угодно разных папок, внутри которых могут быть папки to_print_dir и to_combine_dir
#Если пользователь выберет чекбокс удалить to_print_dir/to_combine_dir, то удалится должны все файлы во всех папках to_print_dir/to_combine_dir

#ChatGPT, напиши код, который очищает выбранные папки

# Функция для очистки указанной директории
def clear_directory(directory):
    if os.path.exists(directory):
        for root, dirs, files in os.walk(directory):
            for file in files:
                file_path = os.path.join(root, file)
                os.remove(file_path)
            for dir in dirs:
                dir_path = os.path.join(root, dir)
                shutil.rmtree(dir_path)

# Функция для поиска и очистки всех вложенных папок to_print_dir и to_combine_dir
def clear_nested_directories(base_dir, target_dir_name):
    for root, dirs, files in os.walk(base_dir):
        if target_dir_name in dirs:
            dir_path = os.path.join(root, target_dir_name)
            clear_directory(dir_path)

# Очистка to_print_dir, если флаг установлен
if clear__to_print_dir :
    clear_nested_directories(output_dir, 'to_print')

# Очистка to_combine_dir, если флаг установлен
if lear__to_combine_dir:
    clear_nested_directories(output_dir, 'to_combine')

# Пример удаления директории pdf_dir, если флаг установлен (если имеется отдельная папка)
if clear__pdf_dir:
    clear_directory(pdf_dir)

In [4]:
def create_dir_if_it_doesnt_exist(root_dir, dir):
    path = os.path.join(root_dir, dir,)
        
    if not os.path.exists(path):
        os.makedirs(path)
        
create_dir_if_it_doesnt_exist('./', input_dir)
create_dir_if_it_doesnt_exist('./', pdf_dir)
create_dir_if_it_doesnt_exist('./', output_dir)


In [5]:
img_half_empty = './img/empty_half.png'

# Убедитесь, что путь к изображению корректный и изображение загружено
img_template_path=''

if note_type == 'cells':
    img_template_path = './img/cells_half.png'
else:
    img_template_path = './img/lines_half.png'


if not os.path.exists(img_template_path):
    raise FileNotFoundError(f"Изображение не найдено по пути: {img_template_path}")

# Открываем изображение с клетками img_half_empty
#img_template = Image.open(img_half_empty)

img_template = Image.open(img_template_path)

In [6]:
def run_ms_word(word_path):
    # Запускаем Microsoft Word
    try:
        is_ms_word_closed = subprocess.Popen([word_path])
        print("Microsoft Word запущен.")
    except Exception as e:
        print(f"Не удалось запустить Microsoft Word: {e}")
        

In [7]:
def is_ms_word_running(process_name="WINWORD.EXE"):
    """Проверяет, работает ли Microsoft Word и полностью ли он загружен."""
    try:
        # Получаем список процессов и проверяем наличие Microsoft Word
        output = subprocess.check_output(["tasklist"], text=True)
        
        if process_name in output:
            print("MS Word работает.")
            
            # Дожидаемся полной загрузки Word
            while True:
                try:
                    # Проверяем, доступен ли объект Word
                    word_app = win32com.client.GetActiveObject("Word.Application")
                    if word_app is not None:
                        print("MS Word полностью загружен и готов к работе.")
                        return True
                except Exception:
                    # Если объект недоступен, продолжаем ожидание
                    print("Ожидание полной загрузки MS Word...")
                    time.sleep(1)  # Ждем 1 секунду перед следующей проверкой
            
        else:
            print("MS Word не запущен.")
            return False
    except Exception as e:
        print(f"Ошибка при проверке процессов: {e}")
        return False
    

In [ ]:
for filename in os.listdir(input_dir):
    if ".docx" in filename.lower() or ".docm" in filename.lower() or ".doc" in filename.lower():
        if "$" not in filename:
            print(filename)
            
            filename_no_ext = filename[:-5].strip()
            temp_pdf_path = (f"{pdf_dir}/{filename_no_ext}.pdf")
            

            # Если Word запущен через subroccess:
            if is_ms_word_running():
                # Microsoft Word не успевает запуститься до запуска этой строки. Как исправить?
                convert( (f"{input_dir}/{filename}"), temp_pdf_path)
            
            
            #Иначе запускаем его и 
            else:
                run_ms_word(word_path)
                # Как здесь дождаться когда Microsoft запустится? После этого нужно выполнить этот код:
                

                # Ожидаем, пока Microsoft Word не запустится
                while not is_ms_word_running():
                    print("Ожидание запуска Microsoft Word...")
                    time.sleep(2)  # Ждем 1 секунду перед следующей проверкой

                # После того как Microsoft Word запущен, выполняем конвертацию
                try:
                    convert(os.path.join(input_dir, filename), temp_pdf_path)
                    print(f'Конвертировано: {input_dir}/{filename} в {temp_pdf_path}')
                except Exception as e:
                    print(f'Ошибка при конвертации: {e}')

    
            print(f"{input_dir}/{filename}")
            

In [ ]:
for input_filename in os.listdir(input_dir):
    input_path = os.path.join(input_dir, input_filename)

    if os.path.isfile(input_path) and '$' not in input_filename:
        
        if ".docx" in filename.lower() or ".docm" in filename.lower() or ".doc" in filename.lower():
            print(input_filename)
            create_dir_if_it_doesnt_exist(output_dir, input_filename[:-5])

In [ ]:
for input_filename in os.listdir(input_dir):
    input_path = os.path.join(input_dir, input_filename)

    if ".docx" in filename.lower() or ".docm" in filename.lower() or ".doc" in filename.lower():
        if os.path.isfile(input_path) and '$' not in input_filename:

            #print(input_filename)
            create_dir_if_it_doesnt_exist(output_dir, input_filename[:-5].strip())

            # Открываем PDF-документ
            pdf_path = os.path.join(pdf_dir, f'{input_filename[:-5].strip()}.pdf')
            
            create_dir_if_it_doesnt_exist(f'{output_dir}/{input_filename[:-5].strip()}', to_combine_dir.strip().replace('./', ''))
            create_dir_if_it_doesnt_exist(f'{output_dir}/{input_filename[:-5].strip()}', to_print_dir.strip().replace('./', ''))

            doc = fitz.open(pdf_path)

            # Перебираем страницы PDF
            for page_num in range(len(doc)):
                # Получаем страницу
                page = doc.load_page(page_num)
                
                # Извлекаем текст с форматированием
                text = page.get_text("text")  # Извлекаем текст с форматированием
            
                # Разбиваем текст на отдельные строки
                text_lines = text.splitlines()
                
                # Создаем копию шаблона для каждой страницы
                #img = img_template.copy()
                #draw = ImageDraw.Draw(img)
                
                
                
                
                
                # Берем высоту буквы по англ. символу "a"
                char_a = 'a' 
                line_height = font.getsize(char_a)[1]  # Получаем высоту строки
                    
                canvas_image_path = './img/cells_half.png'
                line_image_path = './img/line_empty.png'    
                
                # Переносим строки текста на соответствующие линии
                for i, line in enumerate(text_lines):
                    
                    
                                    
                    y_position = (i + 2) * line_spacing - line_height  # Выравниваем низ текста по линии
                    
                    coeff = 20
                    
                    canvas_image = Image.open(canvas_image_path)
                    
                    line_img = Image.open(line_image_path)                
                    draw = ImageDraw.Draw(line_img)
                    
                    #print(f'Рисуем текст на странице {page_num + 1}, строка {i + 1}: {line}')
                    #draw.text((4 * 55, y_position), line, fill=font_color, font=font)
                    draw.text((0,0), line, fill=font_color, font=font)
                    
                    resized_line_img = line_img.resize((1660, line_spacing*font_scale_y__coeff-coeff))
                    
                    
                    #line_img.save(f'{output_dir}/{input_filename[:-5]}/{to_combine_dir}/line {page_num + 1}.png')  # Нумерация страниц начинается с 1
                    
                    #resized_line_img.save(f'{output_dir}/{input_filename[:-5]}/{to_combine_dir}/resized_line {page_num + 1}.png')  # Нумерация страниц начинается с 1


                    #вставить в канвас картинку(строка, координаты, маска картинки)
                    canvas_image.paste(resized_line_img, (4*55, y_position+int(coeff/2)), resized_line_img)

                    
                # Сохраняем результат для текущей страницы
                    canvas_image.save(f'{output_dir}/{input_filename[:-5].strip()}/{to_combine_dir}/{page_num + 1}.png')  # Нумерация страниц начинается с 1
                    
                    
                    #Почему-то вот так обновление путя работает 
                    canvas_image_path=f'{output_dir}/{input_filename[:-5].strip()}/{to_combine_dir}/{page_num + 1}.png'
                    
                    #а так нет:
                    #canvas_image_path=f'{output_dir}/{input_filename[:-5]}/{to_combine_dir}'
                    
                    #В папке to_combine должны быть картинки ТОЛЬКО с цифрами: 1.png, 2.png и т.д.
                    
                    

            # Закрываем PDF-документ
            doc.close()